In [ ]:
# predictive modeling on news
# paper has PR AUC, Precision, Recall, F1 for performance

In [35]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
from sklearn import preprocessing, metrics, model_selection, decomposition #, XBGRegressor
# from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [13]:
# some keywords to try: 
# likely to be fake: cure, hoax, stopped?, Nancy, Pelosi, video, shared, 5G, Melinda, claims, Donald, Gates, Bill, Obama, Facebook 
news=pd.read_csv('allnewsdata.csv')
news

,Unnamed: 0,type,fact_check_url,archive,news_url,news_url2,news_url3,news_url4,news_url5,title,newstitle,content,abstract,publish_date,meta_keywords,month,source,real/fake
0,0,post,https://factcheck.afp.com/false-advice-refusin...,https://perma.cc/J4N6-39D5,https://www.facebook.com/photo.php?fbid=551960...,NaN,NaN,NaN,NaN,Facebook posts shared in at least three countr...,NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake
1,1,article,https://www.politifact.com/factchecks/2020/apr...,NaN,http://legis.wisconsin.gov/assembly/republican...,NaN,NaN,NaN,NaN,Wisconsin is Òclearly seeing a decline in COVI...,"""Wisconsin Legislature Takes Gov. Evers to Court""",speaker robin vos r rochester and senate major...,NaN,NaN,"""""",5,legis.wisconsin.gov,fake
2,2,post,https://factcheck.afp.com/posts-claim-children...,https://perma.cc/V4HX-M2XJ,https://www.facebook.com/iAmJessenia/photos/a....,NaN,NaN,NaN,NaN,Facebook posts claim a child who is infected w...,NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake
3,3,post,https://checkyourfact.com/2020/04/20/fact-chec...,NaN,https://www.facebook.com/kokernagnews/photos/a...,NaN,NaN,NaN,NaN,IndiaÕs Ministry of Home Affairs banning citiz...,NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake
4,5,post,https://checkyourfact.com/2020/04/20/fact-chec...,NaN,https://www.facebook.com/photo.php?fbid=102193...,NaN,NaN,NaN,NaN,"42 Democratic senators, plus two Independents,...",NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5452,10078,article,https://www.politifact.com/factchecks/2020/oct...,NaN,https://www.themonitor.com/2020/10/05/kamala-h...,NaN,NaN,NaN,NaN,Says the “The Rio Grande Valley is 4.7% of the...,"""403 Forbidden""",NaN,NaN,NaN,"""""",11,www.themonitor.com,real
5453,10079,article,https://www.politifact.com/factchecks/2020/oct...,NaN,https://www.wral.com/cooper-forest-engage-in-l...,NaN,NaN,NaN,NaN,"Georgia has “almost 100,000 more (COVID-19) ca...","""Cooper, Forest engage in lone gubernatorial d...",governor dan forest. live debate at u. n c. t ...,democratic gov. roy cooper and republican lt. ...,2020-10-14T18:47:00-04:00,"""Roy Cooper,Dan Forest,2020 governors race,deb...",11,www.wral.com,real
5454,10080,post,https://www.politifact.com/factchecks/2020/oct...,NaN,https://www.facebook.com/photo.php?fbid=402154...,NaN,NaN,NaN,NaN,“Wisconsin Republicans have not passed a singl...,"""Facebook""",NaN,NaN,NaN,"""""",11,www.facebook.com,real
5455,10081,post,https://www.politifact.com/factchecks/2020/sep...,NaN,https://twitter.com/SenChrisLarson/status/1308...,NaN,NaN,NaN,NaN,“(Republicans) have the power to overturn the ...,"""""",this browser is no longer supported. please sw...,NaN,NaN,"""""",11,twitter.com,real


In [14]:
# tokensdist=pd.read_csv('alltokensdistribution.csv')
# # make sure counts and pcts are read as numbers rather than strs

# with pd.option_context("display.max_rows", None):
#     display(tokensdist.sort_values('counts in fake',ascending=False).head(100))


In [15]:
# type(tokensdist.loc[3016,'counts in fake']) # type is str. need remove brackets


In [73]:
# news.where(news.title.str.contains('cure|hoax|stopped|Nancy|Pelosi|video|shared|5G|Melinda|claims|Donald|Gates|Bill|Obama|Facebook',)

In [24]:
news['sw_cure']=np.where(news.title.str.contains('cure'),1,0)
news['sw_hoax']=np.where(news.title.str.contains('hoax'),1,0)
news['sw_stopped']=np.where(news.title.str.contains('stopped'),1,0)
news['sw_Nancy']=np.where(news.title.str.contains('Nancy'),1,0)
news['sw_Pelosi']=np.where(news.title.str.contains('Pelosi'),1,0)
news['sw_video']=np.where(news.title.str.contains('video'),1,0)
news['sw_shared']=np.where(news.title.str.contains('shared'),1,0)
news['sw_5G']=np.where(news.title.str.contains('5G'),1,0)
news['sw_Melinda']=np.where(news.title.str.contains('Melinda'),1,0)
news['sw_claims']=np.where(news.title.str.contains('claims'),1,0)
news['sw_Donald']=np.where(news.title.str.contains('Donald'),1,0)
news['sw_Gates']=np.where(news.title.str.contains('Gates'),1,0)
news['sw_Bill']=np.where(news.title.str.contains('Bill'),1,0)
news['sw_Obama']=np.where(news.title.str.contains('Obama'),1,0)
news['sw_Facebook']=np.where(news.title.str.contains('Facebook'),1,0)
news

,Unnamed: 0,type,fact_check_url,archive,news_url,news_url2,news_url3,news_url4,news_url5,title,newstitle,content,abstract,publish_date,meta_keywords,month,source,real/fake,sw_cure,sw_hoax,sw_stopped,sw_Nancy,sw_Pelosi,sw_video,sw_shared,sw_5G,sw_Melinda,sw_claims,sw_Donald,sw_Gates,sw_Bill,sw_Obama,sw_Facebook
0,0,post,https://factcheck.afp.com/false-advice-refusing-vaccines-circulates-during-covid-19-pandemic,https://perma.cc/J4N6-39D5,https://www.facebook.com/photo.php?fbid=551960552369167&set=a.112646526300574&type=3,NaN,NaN,NaN,NaN,Facebook posts shared in at least three countries as scientists work to develop a COVID-19 vaccine claim to offer a legal way to refuse vaccination.,NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,1,article,https://www.politifact.com/factchecks/2020/apr/21/wisconsin-republican-legislative-leaders/no-wisconsin-not-clearly-seeing-decline-covid-case/,NaN,http://legis.wisconsin.gov/assembly/republicans/news/wisconsin-legislature-takes-gov-evers-to-court/,NaN,NaN,NaN,NaN,Wisconsin is Òclearly seeing a decline in COVID infectionsÓ.,"""Wisconsin Legislature Takes Gov. Evers to Court""",speaker robin vos r rochester and senate majority leader scott fitzgerald r juneau released the following statement after filing an original action in the wisconsin state supreme court challenging the governors extension of the safer at home order through may 26th. the public outcry over the safer at home order continues to increase as positive covid cases decrease or remain flat. there s immense frustration regarding the extension as it goes beyond the executive branchs statutory power.,NaN,NaN,"""""",5,legis.wisconsin.gov,fake,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,post,https://factcheck.afp.com/posts-claim-children-covid-19-must-go-hospitals-alone,https://perma.cc/V4HX-M2XJ,https://www.facebook.com/iAmJessenia/photos/a.10151658017176441/10156793962781441/?type=3,NaN,NaN,NaN,NaN,Facebook posts claim a child who is infected with the novel coronavirus will be taken to a hospital unaccompanied.,NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,3,post,https://checkyourfact.com/2020/04/20/fact-check-india-banned-people-posting-coronavirus-social-media/,NaN,https://www.facebook.com/kokernagnews/photos/a.1070663659747172/2181164582030402/?type=3&theater,NaN,NaN,NaN,NaN,IndiaÕs Ministry of Home Affairs banning citizens from posting about the new coronavirus on social media.,NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,post,https://checkyourfact.com/2020/04/20/fact-check-44-senators-vote-against-sending-coronavirus-stimulus-payments/,NaN,https://www.facebook.com/photo.php?fbid=10219317734188369&set=gm.642428883004151&type=3&theater&ifg=1,NaN,NaN,NaN,NaN,"42 Democratic senators, plus two Independents, voted against sending direct payments to Americans as part of the coronavirus economic stimulus package.",NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5452,10078,article,https://www.politifact.com/factchecks/2020/oct/16/vicente-gonzalez/covid-19-fatalities-rio-grande-valley-double-state/,NaN,https://www.themonitor.com/2020/10/05/kamala-harris-husband-makes-campaign-stop-democratic-stronghold-rgv/,NaN,NaN,NaN,NaN,"Says the “The Rio Grande Valley is 4.7% of the entire state population, but it represents 17% of the (coronavirus) deaths of our state.""","""403 Forbidden""",NaN,NaN,NaN,"""""",11,www.themonitor.com,real,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5453,10079,article,https://www.politifact.com/factchecks/2020/oct/15/roy-cooper/roy-cooper-compares-georgias-covid-19-numbers-nort/,NaN,https://www.wral.com/cooper-forest-engage-in-lone-gubernatorial-debate/19336788/,NaN,NaN,NaN,NaN,"Georgia has “almost 100,000 more (COVID-19) cases, almost twice as many deaths and almost 600 more nursing home deaths

In [26]:
news.columns

Index(['Unnamed: 0', 'type', 'fact_check_url', 'archive', 'news_url',
       'news_url2', 'news_url3', 'news_url4', 'news_url5', 'title',
       'newstitle', 'content', 'abstract', 'publish_date', 'meta_keywords',
       'month', 'source', 'real/fake', 'sw_cure', 'sw_hoax', 'sw_stopped',
       'sw_Nancy', 'sw_Pelosi', 'sw_video', 'sw_shared', 'sw_5G', 'sw_Melinda',
       'sw_claims', 'sw_Donald', 'sw_Gates', 'sw_Bill', 'sw_Obama',
       'sw_Facebook', 'y'],
      dtype='object')

In [31]:
news['y']=np.where(news['real/fake']=='fake',1,0)
news

,Unnamed: 0,type,fact_check_url,archive,news_url,news_url2,news_url3,news_url4,news_url5,title,newstitle,content,abstract,publish_date,meta_keywords,month,source,real/fake,sw_cure,sw_hoax,sw_stopped,sw_Nancy,sw_Pelosi,sw_video,sw_shared,sw_5G,sw_Melinda,sw_claims,sw_Donald,sw_Gates,sw_Bill,sw_Obama,sw_Facebook,y
0,0,post,https://factcheck.afp.com/false-advice-refusing-vaccines-circulates-during-covid-19-pandemic,https://perma.cc/J4N6-39D5,https://www.facebook.com/photo.php?fbid=551960552369167&set=a.112646526300574&type=3,NaN,NaN,NaN,NaN,Facebook posts shared in at least three countries as scientists work to develop a COVID-19 vaccine claim to offer a legal way to refuse vaccination.,NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
1,1,article,https://www.politifact.com/factchecks/2020/apr/21/wisconsin-republican-legislative-leaders/no-wisconsin-not-clearly-seeing-decline-covid-case/,NaN,http://legis.wisconsin.gov/assembly/republicans/news/wisconsin-legislature-takes-gov-evers-to-court/,NaN,NaN,NaN,NaN,Wisconsin is Òclearly seeing a decline in COVID infectionsÓ.,"""Wisconsin Legislature Takes Gov. Evers to Court""",speaker robin vos r rochester and senate majority leader scott fitzgerald r juneau released the following statement after filing an original action in the wisconsin state supreme court challenging the governors extension of the safer at home order through may 26th. the public outcry over the safer at home order continues to increase as positive covid cases decrease or remain flat. there s immense frustration regarding the extension as it goes beyond the executive branchs statutory power.,NaN,NaN,"""""",5,legis.wisconsin.gov,fake,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,post,https://factcheck.afp.com/posts-claim-children-covid-19-must-go-hospitals-alone,https://perma.cc/V4HX-M2XJ,https://www.facebook.com/iAmJessenia/photos/a.10151658017176441/10156793962781441/?type=3,NaN,NaN,NaN,NaN,Facebook posts claim a child who is infected with the novel coronavirus will be taken to a hospital unaccompanied.,NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
3,3,post,https://checkyourfact.com/2020/04/20/fact-check-india-banned-people-posting-coronavirus-social-media/,NaN,https://www.facebook.com/kokernagnews/photos/a.1070663659747172/2181164582030402/?type=3&theater,NaN,NaN,NaN,NaN,IndiaÕs Ministry of Home Affairs banning citizens from posting about the new coronavirus on social media.,NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,5,post,https://checkyourfact.com/2020/04/20/fact-check-44-senators-vote-against-sending-coronavirus-stimulus-payments/,NaN,https://www.facebook.com/photo.php?fbid=10219317734188369&set=gm.642428883004151&type=3&theater&ifg=1,NaN,NaN,NaN,NaN,"42 Democratic senators, plus two Independents, voted against sending direct payments to Americans as part of the coronavirus economic stimulus package.",NaN,NaN,NaN,NaN,NaN,5,www.facebook.com,fake,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5452,10078,article,https://www.politifact.com/factchecks/2020/oct/16/vicente-gonzalez/covid-19-fatalities-rio-grande-valley-double-state/,NaN,https://www.themonitor.com/2020/10/05/kamala-harris-husband-makes-campaign-stop-democratic-stronghold-rgv/,NaN,NaN,NaN,NaN,"Says the “The Rio Grande Valley is 4.7% of the entire state population, but it represents 17% of the (coronavirus) deaths of our state.""","""403 Forbidden""",NaN,NaN,NaN,"""""",11,www.themonitor.com,real,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5453,10079,article,https://www.politifact.com/factchecks/2020/oct/15/roy-cooper/roy-cooper-compares-georgias-covid-19-numbers-nort/,NaN,https://www.wral.com/cooper-forest-engage-in-lone-gubernatorial-debate/19336788/,NaN,NaN,NaN,NaN,"Georgia has “almost 100,000 more (COVID-19) cases, almost twice as many deaths and almost 600 more n

In [36]:
# import statsmodels.api as sm
# import statsmodels.formula.api as smf
# x=news[['sw_cure', 'sw_hoax', 'sw_stopped',
#        'sw_Pelosi', 'sw_video', 'sw_shared', 'sw_5G', 'sw_Melinda', #'sw_Nancy', 
#        'sw_claims', 'sw_Donald',  'sw_Obama', 'sw_Bill', #,'sw_Gates',
#        'sw_Facebook']]
# y=news[['y']]
# log_reg = sm.Logit(y, x).fit()
# print(log_reg.summary())

In [39]:
from sklearn.model_selection import train_test_split
X=news[['sw_cure', 'sw_hoax', 'sw_stopped','sw_Pelosi', 'sw_video', 
           'sw_shared', 'sw_5G', 'sw_Melinda','sw_claims', 'sw_Donald', 
           'sw_Obama','sw_Bill','sw_Facebook']]
y=news[['y']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)
# train_data, val_data = model_selection.train_test_split(data,test_size=0.2)
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)

/Users/yapinghe/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [40]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix 
# top left is true negative, bottom right is true positive, bottom left is false negative, top right is false positive
# x axis is actual lable, y axis is predicted

array([[1119,    5],
       [ 185,   56]])

In [69]:
# precision = TruePositives/(TruePositives + FalsePositives)
56/(56+5)

0.9180327868852459

In [43]:
from sklearn.metrics import confusion_matrix, accuracy_score
accuracy_score(y_test, y_pred)

0.8608058608058609

In [46]:
y_test.sum()

y    241
dtype: int64

In [47]:
len(y_test)

1365

In [72]:
# X_train

In [51]:
y_test.sum(),len(y_test),y_test.sum()/len(y_test)

(y    241
 dtype: int64,
 1365,
 y    0.176557
 dtype: float64)

In [ ]:
# notes from class:
# ROC curves vs. Precision/Recall curves
# prefer ROC curves when the two binary value cases are relatively balanced in the dataset
# if the cases are unbalanced, we don't really care too much about predicting negatives
# for example, if 0 = low risk and 1 = high risk, we may be willing to miss many low risk cases as long as we can correctly identify most high risk cases
# in that case, prefer the model with the right mix of high precision and high recall
# prefer precision/recall curves when the positive value cases are far fewer than the negative value cases
# note that the calculations for precision and recall do not consider the true negatives

# https://neptune.ai/blog/f1-score-accuracy-roc-auc-pr-auc

In [59]:
y_score = logreg.predict_proba(X_test)[:, 1]
y_score

array([0.13843514, 0.13843514, 0.13843514, ..., 0.13843514, 0.13843514,
       0.13843514])

In [62]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score, precision_recall_curve,auc
average_precision = average_precision_score(y_test, y_score)
print(average_precision) # PR AUC

0.3597217361327164


In [71]:
from sklearn.metrics import precision_score, recall_score, f1_score
[precision_score(y_test, y_pred),recall_score(y_test, y_pred),f1_score(y_test,y_pred)]

[0.9180327868852459, 0.23236514522821577, 0.37086092715231794]

In [74]:
56/(56+185)

0.23236514522821577